In [7]:
import time
import warnings
import sys
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
import pandas as pd
import numpy as np
from sklearn.feature_selection import SelectPercentile, f_classif, SelectFromModel
from sklearn.metrics import roc_auc_score, confusion_matrix, precision_recall_curve, auc, mean_squared_error, \
    r2_score, mean_absolute_error,cohen_kappa_score,accuracy_score,f1_score,matthews_corrcoef,precision_score,recall_score
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor, XGBClassifier
import multiprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from lightgbm import LGBMClassifier
from lightgbm import LGBMRegressor

start = time.time()
warnings.filterwarnings("ignore")


def standardize(col):
    return (col - np.mean(col)) / np.std(col)


# the metrics for classification
def statistical(y_true, y_pred, y_pro):
    c_mat = confusion_matrix(y_true, y_pred)
    tn, fp, fn, tp = list(c_mat.flatten())
    se = tp / (tp + fn)
    sp = tn / (tn + fp)
    auc_prc = auc(precision_recall_curve(y_true, y_pro, pos_label=1)[1],
                  precision_recall_curve(y_true, y_pro, pos_label=1)[0])
    acc = (tp + tn) / (tn + fp + fn + tp)
#     acc_skl = accuracy_score(y_true, y_pred)
    auc_roc = roc_auc_score(y_true, y_pro)
    recall = se
#     recall_skl = recall_score(y_true, y_pred)
    precision = tp / (tp + fp)
#     precision_skl = precision_score(y_true, y_pred)
    f1 = 2 * (precision * recall) / (precision + recall) # F1 = 2 * (precision * recall) / (precision + recall)
#     f1_skl = f1_score(y_true, y_pred)
    kappa = cohen_kappa_score(y_true,y_pred)
    mcc = (tp * tn - fp * fn) / np.sqrt((tp + fp) * (tp + fn) * (tn + fp) * (tn + fn) + 1e-8)
#     mcc_skl = matthews_corrcoef(y_true,y_pred)
    return tn,fp,fn,tp,se,sp,auc_prc,acc,auc_roc,recall,precision,f1,kappa,mcc

def all_one_zeros(data):
    if (len(np.unique(data)) == 2):
        flag = False
    else:
        flag = True
    return flag


feature_selection = False
tasks_dic = {'0-CF-2274-desc-split.csv': ['activity']}

In [8]:
file_name = '0-CF-2274-desc-split.csv'
task_type = 'cla'  # 'reg' or 'cla'
dataset_label = file_name.split('/')[-1].split('_')[0]
tasks = tasks_dic[dataset_label]
OPT_ITERS = 50
repetitions = 50
num_pools = 5
unbalance = True
patience = 100
space_ = {'num_leaves': hp.choice('num_leaves', range(100, 300,10)),
          'learning_rate': hp.uniform('learning_rate', 0.005, 0.3),
          'max_depth': hp.choice('max_depth', range(3, 12)),
          'n_estimators': hp.choice('n_estimators', [100,200,300, 400, 500, 1000]),
          'min_child_samples': hp.choice('min_child_samples', range(0, 100,10)),
          'max_bin': hp.choice('max_bin', range(300, 400,10))
          }
num_leaves_ls = range(100, 300,10)
max_depth_ls = range(3, 12)
n_estimators_ls = [100,200,300, 400, 500, 1000]
min_child_samples_ls = range(0, 100,10)
max_bin_ls = range(300, 400,10)
dataset = pd.read_csv(file_name)
pd_res = []

In [ ]:
def hyper_runing(subtask):
    cols = [subtask]
    cols.extend(dataset.columns[(len(tasks) + 1):])
    sub_dataset = dataset[cols]

    # detect the na in the subtask (y cloumn)
    rm_index = sub_dataset[subtask][sub_dataset[subtask].isnull()].index
    sub_dataset.drop(index=rm_index, inplace=True)

    # remove the features with na
    # *******************
    # demension reduction
    # *******************
    # Removing features with low variance
    # threshold = 0.05
    data_fea_var = sub_dataset.iloc[:, 2:].var()
    del_fea1 = list(data_fea_var[data_fea_var <= 0.05].index)
    sub_dataset.drop(columns=del_fea1, inplace=True)

    # pair correlations
    # threshold = 0.95
    data_fea_corr = sub_dataset.iloc[:, 2:].corr()
    del_fea2_col = []
    del_fea2_ind = []
    length = data_fea_corr.shape[1]
    for i in range(length):
        for j in range(i + 1, length):
            if abs(data_fea_corr.iloc[i, j]) >= 0.95:
                del_fea2_col.append(data_fea_corr.columns[i])
                del_fea2_ind.append(data_fea_corr.index[j])
    sub_dataset.drop(columns=del_fea2_ind, inplace=True)

    # standardize the features
    cols_ = list(sub_dataset.columns)[2:]
    sub_dataset[cols_] = sub_dataset[cols_].apply(standardize, axis=0)

    # get the attentivefp data splits
    data_tr = sub_dataset[sub_dataset['group'] == 'train']
    data_va = sub_dataset[sub_dataset['group'] == 'valid']
    data_te = sub_dataset[sub_dataset['group'] == 'test']

    # prepare data for training
    # training set
    data_tr_y = data_tr[subtask].values.reshape(-1, 1)
    data_tr_x = np.array(data_tr.iloc[:, 2:].values)

    # validation set
    data_va_y = data_va[subtask].values.reshape(-1, 1)
    data_va_x = np.array(data_va.iloc[:, 2:].values)

    # test set
    data_te_y = data_te[subtask].values.reshape(-1, 1)
    data_te_x = np.array(data_te.iloc[:, 2:].values)

    if feature_selection:
        # univariate feature selection
        trans1 = SelectPercentile(f_classif, percentile=80)
        trans1.fit(data_tr_x, data_tr_y)
        data_tr_x = trans1.transform(data_tr_x)
        data_va_x = trans1.transform(data_va_x)
        data_te_x = trans1.transform(data_te_x)

        # select from model
        clf = XGBClassifier(n_jobs=12, random_state=1)
        clf = clf.fit(data_tr_x, data_tr_y)
        trans2 = SelectFromModel(clf, prefit=True)

        data_tr_x = trans2.transform(data_tr_x)
        data_va_x = trans2.transform(data_va_x)
        data_te_x = trans2.transform(data_te_x)

    num_fea = data_tr_x.shape[1]
    print('the num of retained features for the ' + dataset_label + ' ' + subtask + ' is:', num_fea)

    def hyper_opt(args):
        model = LGBMClassifier(**args, n_jobs=6, random_state=1,
                              is_unbalance = unbalance) if task_type == 'cla' else LGBMRegressor(**args, n_jobs=6,
                                                                                                   random_state=1)

        model.fit(data_tr_x, data_tr_y, eval_metric='auc' if task_type == 'cla' else 'rmse',
                  eval_set=[(data_va_x, data_va_y)],
                  early_stopping_rounds=patience, verbose=False)
        val_preds = model.predict_proba(data_va_x) if task_type == 'cla' else \
            model.predict(data_va_x)
        loss = 1 - roc_auc_score(data_va_y, val_preds[:, 1]) if task_type == 'cla' else np.sqrt(
            mean_squared_error(data_va_y, val_preds))
        return {'loss': loss, 'status': STATUS_OK}

    # start hyper-parameters optimization
    trials = Trials()
    best_results = fmin(hyper_opt, space_, algo=tpe.suggest, max_evals=OPT_ITERS, trials=trials, show_progressbar=False)
    print('the best hyper-parameters for ' + dataset_label + ' ' + subtask + ' are:  ', best_results)
    best_model = LGBMClassifier(num_leaves=num_leaves_ls[best_results['num_leaves']],
                                        learning_rate=best_results['learning_rate'],
                                        max_depth=max_depth_ls[best_results['max_depth']],
                                        n_estimators=n_estimators_ls[best_results['n_estimators']],
                                        min_child_samples=min_child_samples_ls[best_results['min_child_samples']],
                                        max_bin=max_bin_ls[best_results['max_bin']],
                                        n_jobs=6, random_state=1, verbose=-1, is_unbalance = unbalance) \
        if task_type == 'cla' else LGBMRegressor(
        num_leaves=num_leaves_ls[best_results['num_leaves']],
        learning_rate=best_results['learning_rate'],
        max_depth=max_depth_ls[best_results['max_depth']],
        n_estimators=n_estimators_ls[best_results['n_estimators']],
        min_child_samples=min_child_samples_ls[best_results['min_child_samples']],
        max_bin=max_bin_ls[best_results['max_bin']],
        n_jobs=6, random_state=1, verbose=-1)  
    
    best_model.fit(data_tr_x, data_tr_y, eval_metric='auc' if task_type == 'cla' else 'rmse',
                   eval_set=[(data_va_x, data_va_y)],
                   early_stopping_rounds=patience, verbose=False)
    num_of_compounds = len(sub_dataset)

    if task_type == 'cla':
        # training set
        tr_pred = best_model.predict_proba(data_tr_x)
        tr_results = [dataset_label, subtask, 'tr', num_fea, num_of_compounds, data_tr_y[data_tr_y == 1].shape[0],
                      data_tr_y[data_tr_y == 0].shape[0],
                      data_tr_y[data_tr_y == 0].shape[0] / data_tr_y[data_tr_y == 1].shape[0],
                      num_leaves_ls[best_results['num_leaves']],
                      best_results['learning_rate'],
                      max_depth_ls[best_results['max_depth']],
                      n_estimators_ls[best_results['n_estimators']],
                      min_child_samples_ls[best_results['min_child_samples']],
                      max_bin_ls[best_results['max_bin']]]
        tr_results.extend(statistical(data_tr_y, np.argmax(tr_pred, axis=1), tr_pred[:, 1]))
        # validation set
        va_pred = best_model.predict_proba(data_va_x)
                      
        va_results = [dataset_label, subtask, 'va', num_fea, num_of_compounds, data_va_y[data_va_y == 1].shape[0],
                      data_va_y[data_va_y == 0].shape[0],
                      data_va_y[data_va_y == 0].shape[0] / data_va_y[data_va_y == 1].shape[0],
                      num_leaves_ls[best_results['num_leaves']],
                      best_results['learning_rate'],
                      max_depth_ls[best_results['max_depth']],
                      n_estimators_ls[best_results['n_estimators']],
                      min_child_samples_ls[best_results['min_child_samples']],
                      max_bin_ls[best_results['max_bin']]]
        va_results.extend(statistical(data_va_y, np.argmax(va_pred, axis=1), va_pred[:, 1]))

        # test set
        te_pred = best_model.predict_proba(data_te_x)
        te_results = [dataset_label, subtask, 'te', num_fea, num_of_compounds, data_te_y[data_te_y == 1].shape[0],
                      data_te_y[data_te_y == 0].shape[0],
                      data_te_y[data_te_y == 0].shape[0] / data_te_y[data_te_y == 1].shape[0],
                      num_leaves_ls[best_results['num_leaves']],
                      best_results['learning_rate'],
                      max_depth_ls[best_results['max_depth']],
                      n_estimators_ls[best_results['n_estimators']],
                      min_child_samples_ls[best_results['min_child_samples']],
                      max_bin_ls[best_results['max_bin']]]
        te_results.extend(statistical(data_te_y, np.argmax(te_pred, axis=1), te_pred[:, 1]))
    else:
        # training set
        tr_pred = best_model.predict(data_tr_x)
        tr_results = [dataset_label, subtask, 'tr', num_fea, num_of_compounds,
                      num_leaves_ls[best_results['num_leaves']],
                      best_results['learning_rate'],
                      max_depth_ls[best_results['max_depth']],
                      n_estimators_ls[best_results['n_estimators']],
                      min_child_samples_ls[best_results['min_child_samples']],
                      max_bin_ls[best_results['max_bin']],
                      np.sqrt(mean_squared_error(data_tr_y, tr_pred)), r2_score(data_tr_y, tr_pred),
                      mean_absolute_error(data_tr_y, tr_pred)]

        # validation set
        va_pred = best_model.predict(data_va_x)
        va_results = [dataset_label, subtask, 'va', num_fea, num_of_compounds,
                      num_leaves_ls[best_results['num_leaves']],
                      best_results['learning_rate'],
                      max_depth_ls[best_results['max_depth']],
                      n_estimators_ls[best_results['n_estimators']],
                      min_child_samples_ls[best_results['min_child_samples']],
                      max_bin_ls[best_results['max_bin']],
                      np.sqrt(mean_squared_error(data_va_y, va_pred)), r2_score(data_va_y, va_pred),
                      mean_absolute_error(data_va_y, va_pred)]

        # test set
        te_pred = best_model.predict(data_te_x)
        te_results = [dataset_label, subtask, 'te', num_fea, num_of_compounds,
                      num_leaves_ls[best_results['num_leaves']],
                      best_results['learning_rate'],
                      max_depth_ls[best_results['max_depth']],
                      n_estimators_ls[best_results['n_estimators']],
                      min_child_samples_ls[best_results['min_child_samples']],
                      max_bin_ls[best_results['max_bin']],
                      np.sqrt(mean_squared_error(data_te_y, te_pred)), r2_score(data_te_y, te_pred),
                      mean_absolute_error(data_te_y, te_pred)]
    return tr_results, va_results, te_results


pool = multiprocessing.Pool(num_pools)
res = pool.starmap(hyper_runing, zip(tasks))
pool.close()
pool.join()
for item in res:
    for i in range(3):
        pd_res.append(item[i])
if task_type == 'cla':
    best_hyper = pd.DataFrame(pd_res, columns=['dataset', 'subtask', 'set',
                                               'num_of_retained_feature',
                                               'num_of_compounds', 'postives',
                                               'negtives', 'negtives/postives',
                                               'num_leaves', 
                                               'learning_rate','max_depth','n_estimators','min_child_samples','max_bin',
                                               'tn', 'fp', 'fn', 'tp', 'se', 'sp',
                                               'auc_prc', 'acc', 'auc_roc','recall','precision','f1','kappa','mcc'])
else:
    best_hyper = pd.DataFrame(pd_res, columns=['dataset', 'subtask', 'set',
                                               'num_leaves', 
                                               'learning_rate','max_depth','n_estimators','min_child_samples','max_bin',
                                               'rmse', 'r2', 'mae'])
best_hyper.to_csv('./stat_res/' + dataset_label + '_moe_pubsub_LGB_hyperopt_info.csv', index=0)

if task_type == 'cla':
    print('train', best_hyper[best_hyper['set'] == 'tr']['auc_roc'].mean(),
          best_hyper[best_hyper['set'] == 'tr']['auc_prc'].mean())
    print('valid', best_hyper[best_hyper['set'] == 'va']['auc_roc'].mean(),
          best_hyper[best_hyper['set'] == 'va']['auc_prc'].mean())
    print('test', best_hyper[best_hyper['set'] == 'te']['auc_roc'].mean(),
          best_hyper[best_hyper['set'] == 'te']['auc_prc'].mean())
else:
    print('train', best_hyper[best_hyper['set'] == 'tr']['rmse'].mean(),
          best_hyper[best_hyper['set'] == 'tr']['r2'].mean(), best_hyper[best_hyper['set'] == 'tr']['mae'].mean())
    print('valid', best_hyper[best_hyper['set'] == 'va']['rmse'].mean(),
          best_hyper[best_hyper['set'] == 'va']['r2'].mean(), best_hyper[best_hyper['set'] == 'va']['mae'].mean())
    print('test', best_hyper[best_hyper['set'] == 'te']['rmse'].mean(),
          best_hyper[best_hyper['set'] == 'te']['r2'].mean(), best_hyper[best_hyper['set'] == 'te']['mae'].mean())
    

In [ ]:
# 50 repetitions based on thr best hypers
dataset.drop(columns=['group'], inplace=True)

In [211]:
pd_res = []
def best_model_runing(split):
    seed = split
    if task_type == 'cla':
        while True:
            training_data, data_te = train_test_split(sub_dataset, test_size=0.1, random_state=seed)
            # the training set was further splited into the training set and validation set
            data_tr, data_va = train_test_split(training_data, test_size=0.1, random_state=seed)
            if (all_one_zeros(data_tr[subtask]) or all_one_zeros(data_va[subtask]) or all_one_zeros(data_te[subtask])):
                print(
                    '\ninvalid random seed {} due to one class presented in the {} splitted sets...'.format(seed,
                                                                                                            subtask))
                print('Changing to another random seed...\n')
                seed = np.random.randint(50, 999999)
            else:
                print('random seed used in repetition {} is {}'.format(split, seed))
                break
    else:
        training_data, data_te = train_test_split(sub_dataset, test_size=0.1, random_state=seed)
        # the training set was further splited into the training set and validation set
        data_tr, data_va = train_test_split(training_data, test_size=0.1, random_state=seed)

    # prepare data for training
    # training set
    data_tr_y = data_tr[subtask].values.reshape(-1, 1)
    data_tr_x = np.array(data_tr.iloc[:, 1:].values)

    # validation set
    data_va_y = data_va[subtask].values.reshape(-1, 1)
    data_va_x = np.array(data_va.iloc[:, 1:].values)

    # test set
    data_te_y = data_te[subtask].values.reshape(-1, 1)
    data_te_x = np.array(data_te.iloc[:, 1:].values)

    if feature_selection:
        # univariate feature selection
        trans1 = SelectPercentile(f_classif, percentile=80)
        trans1.fit(data_tr_x, data_tr_y)
        data_tr_x = trans1.transform(data_tr_x)
        data_va_x = trans1.transform(data_va_x)
        data_te_x = trans1.transform(data_te_x)

        # select from model
        clf = XGBClassifier(n_jobs=6, random_state=1)
        clf = clf.fit(data_tr_x, data_tr_y)
        trans2 = SelectFromModel(clf, prefit=True)

        data_tr_x = trans2.transform(data_tr_x)
        data_va_x = trans2.transform(data_va_x)
        data_te_x = trans2.transform(data_te_x)

    num_fea = data_tr_x.shape[1]
    pos_weight = (len(sub_dataset) - sum(sub_dataset[subtask])) / sum(sub_dataset[subtask])
#     model = LGBMClassifier(num_leaves=best_hyper[best_hyper.subtask == subtask].iloc[0,]['num_leaves'],
#                           learning_rate=best_hyper[best_hyper.subtask == subtask].iloc[0,]['learning_rate'],
#                           max_depth=best_hyper[best_hyper.subtask == subtask].iloc[0,]['max_depth'],
#                           n_estimators=best_hyper[best_hyper.subtask == subtask].iloc[0,]['n_estimators'],
#                           min_child_samples=best_hyper[best_hyper.subtask == subtask].iloc[0,]['min_child_samples'],
#                           n_jobs=6, random_state=1,is_unbalance = unbalance) \

# Set the hyper-parameters based on previous experience .Use the above comment code if you need to tune hyper parameters
    model =LGBMClassifier(
                        boosting_type = 'gbdt',
                        class_weight = None,
                        colsample_bytree = 1.0,
                        importance_type = 'split',
                        learning_rate  = 0.1219922716918981,
                        max_depth  = 15,
                        min_child_samples  = 58,
                        min_child_weight  = 0.001,
                        min_split_gain  = 0.0,
                        n_estimators  = 300,
                        num_leaves  = 189,
                        objective  = None,
                        reg_alpha  = 8.031751803464846e-06,
                        reg_lambda  = 0.0649255805574003,
                        silent  = True,
                        subsample  = 1.0,
                        subsample_for_bin  = 200000,
                        subsample_freq  = 0,
                        bagging_fraction  = 1.0,
                        bagging_freq  = 2,
                        feature_fraction  = 0.7344966859224016,
                          n_jobs=6, random_state=1,
                        ) \
        if task_type == 'cla' else LGBMRegressor(
        num_leaves=best_hyper[best_hyper.subtask == subtask].iloc[0,]['num_leaves'],
        learning_rate=best_hyper[best_hyper.subtask == subtask].iloc[0,]['learning_rate'],
        max_depth=best_hyper[best_hyper.subtask == subtask].iloc[0,]['max_depth'],
        n_estimators=best_hyper[best_hyper.subtask == subtask].iloc[0,]['n_estimators'],
        min_child_samples=best_hyper[best_hyper.subtask == subtask].iloc[0,]['min_child_samples'],
        n_jobs=6, random_state=1, seed=1)

    model.fit(data_tr_x, data_tr_y, eval_metric='auc' if task_type == 'cla' else 'rmse',
              eval_set=[(data_va_x, data_va_y)],
              early_stopping_rounds=patience, verbose=False)
    num_of_compounds = sub_dataset.shape[0]
    if task_type == 'cla':
        # training set
        tr_pred = model.predict_proba(data_tr_x)
        tr_results = [split, dataset_label, subtask, 'tr', num_fea, num_of_compounds,
                      data_tr_y[data_tr_y == 1].shape[0],
                      data_tr_y[data_tr_y == 0].shape[0],
                      data_tr_y[data_tr_y == 0].shape[0] / data_tr_y[data_tr_y == 1].shape[0]]
        tr_results.extend(statistical(data_tr_y, np.argmax(tr_pred, axis=1), tr_pred[:, 1]))

        # validation set
        va_pred = model.predict_proba(data_va_x)
        va_results = [split, dataset_label, subtask, 'va', num_fea, num_of_compounds,
                      data_va_y[data_va_y == 1].shape[0],
                      data_va_y[data_va_y == 0].shape[0],
                      data_va_y[data_va_y == 0].shape[0] / data_va_y[data_va_y == 1].shape[0]]
        va_results.extend(statistical(data_va_y, np.argmax(va_pred, axis=1), va_pred[:, 1]))

        # test set
        te_pred = model.predict_proba(data_te_x)
        te_results = [split, dataset_label, subtask, 'te', num_fea, num_of_compounds,
                      data_te_y[data_te_y == 1].shape[0],
                      data_te_y[data_te_y == 0].shape[0],
                      data_te_y[data_te_y == 0].shape[0] / data_te_y[data_te_y == 1].shape[0]]
        te_results.extend(statistical(data_te_y, np.argmax(te_pred, axis=1), te_pred[:, 1]))
    else:
        # training set
        tr_pred = model.predict(data_tr_x)
        tr_results = [split, dataset_label, subtask, 'tr', num_fea, num_of_compounds,
                      np.sqrt(mean_squared_error(data_tr_y, tr_pred)), r2_score(data_tr_y, tr_pred),
                      mean_absolute_error(data_tr_y, tr_pred)]

        # validation set
        va_pred = model.predict(data_va_x)
        va_results = [split, dataset_label, subtask, 'va', num_fea, num_of_compounds,
                      np.sqrt(mean_squared_error(data_va_y, va_pred)), r2_score(data_va_y, va_pred),
                      mean_absolute_error(data_va_y, va_pred)]

        # test set
        te_pred = model.predict(data_te_x)
        te_results = [split, dataset_label, subtask, 'te', num_fea, num_of_compounds,
                      np.sqrt(mean_squared_error(data_te_y, te_pred)), r2_score(data_te_y, te_pred),
                      mean_absolute_error(data_te_y, te_pred)]
    return tr_results, va_results, te_results


for subtask in tasks:
    cols = [subtask]
    cols.extend(dataset.columns[(len(tasks) + 1):])
    # cols.extend(dataset.columns[(617+1):])
    sub_dataset = dataset[cols]

    # detect the NA in the subtask (y cloumn)
    rm_index = sub_dataset[subtask][sub_dataset[subtask].isnull()].index
    sub_dataset.drop(index=rm_index, inplace=True)

    # remove the features with na
    if dataset_label != 'hiv':
        sub_dataset = sub_dataset.dropna(axis=1)
    else:
        sub_dataset = sub_dataset.dropna(axis=0)

    # *******************
    # demension reduction
    # *******************
    # Removing features with low variance
    # threshold = 0.05
    data_fea_var = sub_dataset.iloc[:, 1:].var()
    del_fea1 = list(data_fea_var[data_fea_var <= 0.05].index)
    sub_dataset.drop(columns=del_fea1, inplace=True)

    # pair correlations
    # threshold = 0.95
    data_fea_corr = sub_dataset.iloc[:, 1:].corr()
    del_fea2_col = []
    del_fea2_ind = []
    length = data_fea_corr.shape[1]
    for i in range(length):
        for j in range(i + 1, length):
            if abs(data_fea_corr.iloc[i, j]) >= 0.95:
                del_fea2_col.append(data_fea_corr.columns[i])
                del_fea2_ind.append(data_fea_corr.index[j])
    sub_dataset.drop(columns=del_fea2_ind, inplace=True)

    # standardize the features
    cols_ = list(sub_dataset.columns)[1:]
    sub_dataset[cols_] = sub_dataset[cols_].apply(standardize, axis=0)

    # for split in range(1, splits+1):
    pool = multiprocessing.Pool(num_pools)
    res = pool.starmap(best_model_runing, zip(range(1, repetitions + 1)))
    pool.close()
    pool.join()
    for item in res:
        for i in range(3):
            pd_res.append(item[i])
if task_type == 'cla':
    stat_res = pd.DataFrame(pd_res, columns=['split', 'dataset', 'subtask', 'set',
                                             'num_of_retained_feature',
                                             'num_of_compounds', 'postives',
                                             'negtives', 'negtives/postives',
                                             'tn', 'fp', 'fn', 'tp', 'se', 'sp',
                                             'auc_prc', 'acc', 'auc_roc','recall','precision','f1','kappa','mcc'])
else:
    stat_res = pd.DataFrame(pd_res, columns=['split', 'dataset', 'subtask', 'set',
                                             'num_of_retained_feature',
                                             'num_of_compounds', 'rmse', 'r2', 'mae'])
stat_res.to_csv('./stat_res/' + dataset_label + '_xgb_statistical_results_split50_20220820.csv', index=0)
# single tasks
if len(tasks) == 1:
    args = {'data_label': dataset_label, 'metric': 'auc_roc' if task_type == 'cla' else 'rmse', 'model': 'XGB'}
    print('{}_{}: the mean {} for the training set is {:.3f} with std {:.3f}'.format(args['data_label'], args['model'],
                                                                                     args['metric'], np.mean(
            stat_res[stat_res['set'] == 'tr'][args['metric']]), np.std(
            stat_res[stat_res['set'] == 'tr'][args['metric']])))
    print(
        '{}_{}: the mean {} for the validation set is {:.3f} with std {:.3f}'.format(args['data_label'], args['model'],
                                                                                     args['metric'], np.mean(
                stat_res[stat_res['set'] == 'va'][args['metric']]), np.std(
                stat_res[stat_res['set'] == 'va'][args['metric']])))
    print('{}_{}: the mean {} for the test set is {:.3f} with std {:.3f}'.format(args['data_label'], args['model'],
                                                                                 args['metric'], np.mean(
            stat_res[stat_res['set'] == 'te'][args['metric']]), np.std(
            stat_res[stat_res['set'] == 'te'][args['metric']])))
# multi-tasks
else:
    args = {'data_label': dataset_label, 'metric': 'auc_roc' if dataset_label != 'muv' else 'auc_prc', 'model': 'LGB'}
    tr_acc = np.zeros(repetitions)
    va_acc = np.zeros(repetitions)
    te_acc = np.zeros(repetitions)
    for subtask in tasks:
        tr = stat_res[stat_res['set'] == 'tr']
        tr_acc = tr_acc + tr[tr['subtask'] == subtask][args['metric']].values

        va = stat_res[stat_res['set'] == 'va']
        va_acc = va_acc + va[va['subtask'] == subtask][args['metric']].values

        te = stat_res[stat_res['set'] == 'te']
        te_acc = te_acc + te[te['subtask'] == subtask][args['metric']].values
    tr_acc = tr_acc / len(tasks)
    va_acc = va_acc / len(tasks)
    te_acc = te_acc / len(tasks)
    print('{}_{}: the mean {} for the training set is {:.3f} with std {:.3f}'.format(args['data_label'], args['model'],
                                                                                     args['metric'], np.mean(tr_acc),
                                                                                     np.std(tr_acc)))
    print(
        '{}_{}: the mean {} for the validation set is {:.3f} with std {:.3f}'.format(args['data_label'], args['model'],
                                                                                     args['metric'], np.mean(va_acc),
                                                                                     np.std(va_acc)))
    print('{}_{}: the mean {} for the test set is {:.3f} with std {:.3f}'.format(args['data_label'], args['model'],
                                                                                 args['metric'], np.mean(te_acc),
                                                                                 np.std(te_acc)))
end = time.time()  # get the end time
print('the total elapsed time is:', (end - start), 'S')

random seed used in repetition 13 is 13
random seed used in repetition 1 is 1
random seed used in repetition 10 is 10
random seed used in repetition 4 is 4
random seed used in repetition 7 is 7
[LightGBM] [Warning] bagging_fraction is set=1.0, subsample=1.0 will be ignored. Current value: bagging_fraction=1.0
[LightGBM] [Warning] bagging_fraction is set=1.0, subsample=1.0 will be ignored. Current value: bagging_fraction=1.0
[LightGBM] [Warning] bagging_fraction is set=1.0, subsample=1.0 will be ignored. Current value: bagging_fraction=1.0
[LightGBM] [Warning] bagging_fraction is set=1.0, subsample=1.0 will be ignored. Current value: bagging_fraction=1.0
[LightGBM] [Warning] feature_fraction is set=0.7344966859224016, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7344966859224016
[LightGBM] [Warning] feature_fraction is set=0.7344966859224016, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7344966859224016
[LightGBM] [Warning] bagging_f

[LightGBM] [Warning] bagging_freq is set=2, subsample_freq=0 will be ignored. Current value: bagging_freq=2
random seed used in repetition 25 is 25
[LightGBM] [Warning] bagging_fraction is set=1.0, subsample=1.0 will be ignored. Current value: bagging_fraction=1.0
[LightGBM] [Warning] feature_fraction is set=0.7344966859224016, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7344966859224016
[LightGBM] [Warning] bagging_freq is set=2, subsample_freq=0 will be ignored. Current value: bagging_freq=2
random seed used in repetition 18 is 18
[LightGBM] [Warning] bagging_fraction is set=1.0, subsample=1.0 will be ignored. Current value: bagging_fraction=1.0
[LightGBM] [Warning] feature_fraction is set=0.7344966859224016, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7344966859224016
[LightGBM] [Warning] bagging_freq is set=2, subsample_freq=0 will be ignored. Current value: bagging_freq=2
random seed used in repetition 28 is 28
[LightGBM] [Wa

[LightGBM] [Warning] bagging_fraction is set=1.0, subsample=1.0 will be ignored. Current value: bagging_fraction=1.0
[LightGBM] [Warning] feature_fraction is set=0.7344966859224016, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7344966859224016
[LightGBM] [Warning] feature_fraction is set=0.7344966859224016, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7344966859224016
[LightGBM] [Warning] bagging_freq is set=2, subsample_freq=0 will be ignored. Current value: bagging_freq=2
[LightGBM] [Warning] bagging_freq is set=2, subsample_freq=0 will be ignored. Current value: bagging_freq=2
random seed used in repetition 49 is 49
[LightGBM] [Warning] bagging_fraction is set=1.0, subsample=1.0 will be ignored. Current value: bagging_fraction=1.0
[LightGBM] [Warning] feature_fraction is set=0.7344966859224016, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7344966859224016
[LightGBM] [Warning] bagging_freq is set=2, subs

In [212]:
# acc auc_roc recall precision f1 kappa mcc
acc_str = 'acc of training set is {:.3f}±{:.3f}, validation set is {:.3f}±{:.3f}, test set is {:.3f}±{:.3f}'.format(
                np.mean(stat_res[stat_res['set'] == 'tr']['acc']), 
                np.std(stat_res[stat_res['set'] == 'tr']['acc']),
                np.mean(stat_res[stat_res['set'] == 'va']['acc']), 
                np.std(stat_res[stat_res['set'] == 'va']['acc']),
                np.mean(stat_res[stat_res['set'] == 'te']['acc']), 
                np.std(stat_res[stat_res['set'] == 'te']['acc']),
)
auc_str = 'auc_roc of training set is {:.3f}±{:.3f}, validation set is {:.3f}±{:.3f}, test set is {:.3f}±{:.3f}'.format(
                np.mean(stat_res[stat_res['set'] == 'tr']['auc_roc']), 
                np.std(stat_res[stat_res['set'] == 'tr']['auc_roc']),
                np.mean(stat_res[stat_res['set'] == 'va']['auc_roc']), 
                np.std(stat_res[stat_res['set'] == 'va']['auc_roc']),
                np.mean(stat_res[stat_res['set'] == 'te']['auc_roc']), 
                np.std(stat_res[stat_res['set'] == 'te']['auc_roc']),
)
recall_str = 'recall of training set is {:.3f}±{:.3f}, validation set is {:.3f}±{:.3f}, test set is {:.3f}±{:.3f}'.format(
                np.mean(stat_res[stat_res['set'] == 'tr']['recall']), 
                np.std(stat_res[stat_res['set'] == 'tr']['recall']),
                np.mean(stat_res[stat_res['set'] == 'va']['recall']), 
                np.std(stat_res[stat_res['set'] == 'va']['recall']),
                np.mean(stat_res[stat_res['set'] == 'te']['recall']), 
                np.std(stat_res[stat_res['set'] == 'te']['recall']),
)
precision_str = 'precision of training set is {:.3f}±{:.3f}, validation set is {:.3f}±{:.3f}, test set is {:.3f}±{:.3f}'.format(
                np.mean(stat_res[stat_res['set'] == 'tr']['precision']), 
                np.std(stat_res[stat_res['set'] == 'tr']['precision']),
                np.mean(stat_res[stat_res['set'] == 'va']['precision']), 
                np.std(stat_res[stat_res['set'] == 'va']['precision']),
                np.mean(stat_res[stat_res['set'] == 'te']['precision']), 
                np.std(stat_res[stat_res['set'] == 'te']['precision']),
)
f1_str = 'f1 of training set is {:.3f}±{:.3f}, validation set is {:.3f}±{:.3f}, test set is {:.3f}±{:.3f}'.format(
                np.mean(stat_res[stat_res['set'] == 'tr']['f1']), 
                np.std(stat_res[stat_res['set'] == 'tr']['f1']),
                np.mean(stat_res[stat_res['set'] == 'va']['f1']), 
                np.std(stat_res[stat_res['set'] == 'va']['f1']),
                np.mean(stat_res[stat_res['set'] == 'te']['f1']), 
                np.std(stat_res[stat_res['set'] == 'te']['f1']),
)
kappa_str = 'kappa of training set is {:.3f}±{:.3f}, validation set is {:.3f}±{:.3f}, test set is {:.3f}±{:.3f}'.format(
                np.mean(stat_res[stat_res['set'] == 'tr']['kappa']), 
                np.std(stat_res[stat_res['set'] == 'tr']['kappa']),
                np.mean(stat_res[stat_res['set'] == 'va']['kappa']), 
                np.std(stat_res[stat_res['set'] == 'va']['kappa']),
                np.mean(stat_res[stat_res['set'] == 'te']['kappa']), 
                np.std(stat_res[stat_res['set'] == 'te']['kappa']),
)
mcc_str = 'mcc of training set is {:.3f}±{:.3f}, validation set is {:.3f}±{:.3f}, test set is {:.3f}±{:.3f}'.format(
                np.mean(stat_res[stat_res['set'] == 'tr']['mcc']), 
                np.std(stat_res[stat_res['set'] == 'tr']['mcc']),
                np.mean(stat_res[stat_res['set'] == 'va']['mcc']), 
                np.std(stat_res[stat_res['set'] == 'va']['mcc']),
                np.mean(stat_res[stat_res['set'] == 'te']['mcc']), 
                np.std(stat_res[stat_res['set'] == 'te']['mcc']),
)
print('the elapsed time is:', (end - start)/3600, 'H')

the elapsed time is: 0.7360520495971044 H


In [213]:
print(acc_str)
print(auc_str)
print(recall_str)
print(precision_str)
print(f1_str)
print(kappa_str)
print(mcc_str)

acc of training set is 0.964±0.036, validation set is 0.874±0.023, test set is 0.867±0.023
auc_roc of training set is 0.991±0.015, validation set is 0.878±0.030, test set is 0.864±0.031
recall of training set is 0.821±0.193, validation set is 0.440±0.089, test set is 0.429±0.102
precision of training set is 0.976±0.020, validation set is 0.757±0.099, test set is 0.739±0.112
f1 of training set is 0.877±0.159, validation set is 0.549±0.089, test set is 0.532±0.106
kappa of training set is 0.859±0.170, validation set is 0.483±0.091, test set is 0.462±0.105
mcc of training set is 0.869±0.149, validation set is 0.510±0.082, test set is 0.489±0.097


In [215]:
with open('output_lgb_20220906.txt', 'w') as f:
    f.write(acc_str+'\n')
    f.write(auc_str+'\n')
    f.write(recall_str+'\n')
    f.write(precision_str+'\n')
    f.write(f1_str+'\n')
    f.write(kappa_str+'\n')
    f.write(mcc_str+'\n')